In [1]:
import asyncio
from typing import List, Dict, Optional
import json
import time
from pathlib import Path

import aiohttp
import numpy as np
import pandas as pd
import pendulum
import logging
import requests
from dataclasses import dataclass
from enum import Enum

from sqlalchemy import create_engine, select, text, and_

from sqlalchemy.orm import sessionmaker, declarative_base
from utils.utils import LOCAL_AIRFLOW_PG_URI, LOCAL_PG_URI

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
logging.basicConfig(level=logging.INFO)

In [4]:
Base = declarative_base()

In [5]:
engine = create_engine(LOCAL_AIRFLOW_PG_URI)
SessionLocal = sessionmaker(bind=engine)

## Note:
See `240220_sl_surf_spots.ipynb` for spot getter

In [6]:
response = requests.get("https://services.surfline.com/taxonomy?type=taxonomy&id=58f7ed51dadb30820bb3879c&maxDepth=0")

* You will not get Surfline forecast data without a valid Surfline premium login. Add your credentials to `.env.development`:
  ```
  SURFLINE_EMAIL=xxx
  SURFLINE_PASSWORD=yyy
  ```

##### Requests

`https://services.surfline.com/kbyg/spots/forecasts/{type}?{params}`


Type|Data
----|----
rating|array of human-readable and numeric (0-6) ratings
wave|array of min/max sizes & optimal scores
wind|array of wind directions/speeds & optimal scores
tides|array of types & heights
weather|array of sunrise/set times, array of temperatures/weather conditions

Param|Values|Effect
-----|------|------
spotId|string|Surfline spot id that you want data for. A typical Surfline URL is `https://www.surfline.com/surf-report/venice-breakwater/590927576a2e4300134fbed8` where `590927576a2e4300134fbed8` is the `spotId`
days|integer|Number of forecast days to get (Max 6 w/o access token, Max 17 w/ premium token)
intervalHours|integer|Minimum of 1 (hour)
maxHeights|boolean|`true` seems to remove min & optimal values from the wave data output
sds|boolean|If true, use the new LOTUS forecast engine
accesstoken|string|Auth token to get premium data access (optional)

Anywhere there is an `optimalScore` the value can be interpreted as follows:

Value|Meaning
-----|-------
0|Suboptimal
1|Good
2|Optimal


In [7]:
types = ["rating", "wave", "wind", "tides", "weather"]
params = ["spotId", "days", "intervalHours", "maxHeights", "sds", "accesstoken"]
base = "https://services.surfline.com/kbyg/spots/forecasts"

In [8]:
datapath = Path('./data')


In [9]:
df = pd.read_csv(datapath/'spot_list.csv')

In [10]:
df.head()

,Unnamed: 0,ids,names,lon,lat,urls
0,0,584204204e65fad6a7709b5d,Dauphin Island,-88.11700,30.229000,https://www.surfline.com/surf-report/dauphin-i...
1,1,584204204e65fad6a7709b61,Spuds,-87.54900,30.273000,https://www.surfline.com/surf-report/spuds/584...
2,2,584204204e65fad6a7709b62,Alabama Point,-87.56200,30.270000,https://www.surfline.com/surf-report/alabama-p...
3,3,584204204e65fad6a7709b60,West Pass,-87.73700,30.239000,https://www.surfline.com/surf-report/west-pass...
4,4,65948156c329a78a0914a15e,Morgantown Beach,-87.91913,30.230299,https://www.surfline.com/surf-report/morgantow...


Get the spot `id` for 1st Street Jetty in Va Beach

In [11]:
jetty_id = df[df['names'].str.contains('1st Street Jetty', case=False, na=False)]['ids'].values[0]
jetty_id

'584204214e65fad6a7709ce7'

In [12]:
ex_params = {params[0]: jetty_id}
ex_params

{'spotId': '584204214e65fad6a7709ce7'}

Surfline seems to change their spot IDs periodically. Check a spot on the website and pass the objectId from the url as a param to debug if this is the case. If they've changed you'll need to run the notebook `240220_sl_surf_spots.ipynb` as mentioned above to refresh the spots dataset

In [13]:
debug_params = {params[0]: "584204214e65fad6a7709ce7"}

In [14]:
res = requests.get(f"{base}/{types[0]}", params=ex_params)
res.status_code

200

In [15]:
rating_json = res.json()

In [16]:
four_day_json = res.json()
if 'data' in four_day_json and 'rating' in four_day_json['data']:
    four_day_json['data']['rating'] = four_day_json['data']['rating'][:24]

In [17]:
def cull_extra_days(full_json):
    if 'data' in full_json and 'rating' in full_json['data']:
        full_json['data']['rating'] = full_json['data']['rating'][:24]

Drop extra days of forecast

In [18]:
cull_extra_days(four_day_json)

In [19]:
len(four_day_json['data']['rating'])

24

Convert a unix timestamp -> utc

In [20]:
pendulum.from_timestamp(rating_json['data']['rating'][0]['timestamp'], 'UTC')

DateTime(2024, 6, 10, 4, 0, 0, tzinfo=Timezone('UTC'))

In [21]:
pendulum.from_timestamp(rating_json['data']['rating'][int(72 / 3)-1]['timestamp'], 'UTC')

DateTime(2024, 6, 11, 3, 0, 0, tzinfo=Timezone('UTC'))

The `utcOffset` field seems to be aware that I'm working in EST currently. Either that or it's the time coding for the spot itself.

Let's check a west coast spot to confirm how this is handled

In [22]:
df

,Unnamed: 0,ids,names,lon,lat,urls
0,0,584204204e65fad6a7709b5d,Dauphin Island,-88.117000,30.229000,https://www.surfline.com/surf-report/dauphin-i...
1,1,584204204e65fad6a7709b61,Spuds,-87.549000,30.273000,https://www.surfline.com/surf-report/spuds/584...
2,2,584204204e65fad6a7709b62,Alabama Point,-87.562000,30.270000,https://www.surfline.com/surf-report/alabama-p...
3,3,584204204e65fad6a7709b60,West Pass,-87.737000,30.239000,https://www.surfline.com/surf-report/west-pass...
4,4,65948156c329a78a0914a15e,Morgantown Beach,-87.919130,30.230299,https://www.surfline.com/surf-report/morgantow...
...,...,...,...,...,...,...
1298,1298,640a2d14451905376297f483,Rutherford Beach,-93.124300,29.758500,https://www.surfline.com/surf-report/rutherfor...
1299,1299,5842041f4e65fad6a7708a1a,Assateague,-75.177040,38.148058,https://www.surfline.com/surf-report/assateagu...
1300,1300,5842041f4e65fad6a770886d,Ocean City Boardwalk,-75.081170,38.338461,https://www.surfline.com/surf-report/ocean-cit...
1301,1301,5842041f4e65fad6a7708a1b,North End to Ocean City Inlet,-75.080177,38.338890,https://www.surfline.com/surf-report/north-end...


In [23]:
la_jolla_id = df[df['names'].str.contains("La Jolla", case=False, na=False)]['ids'].values[0]
la_jolla_dict = {params[0]: la_jolla_id}

In [24]:
la_jolla_dict

{'spotId': '5842041f4e65fad6a77088cc'}

In [25]:
pendulum.now("utc")

DateTime(2024, 6, 10, 9, 51, 5, 578016, tzinfo=Timezone('UTC'))

In [26]:
new_dict = {"spot_id": "test", "spot_name": "test_2", "date": pendulum.now("utc"), "forecast": four_day_json}

In [27]:
@dataclass
class SlApiEndpoints(Enum):
    RATING = 'rating'
    WAVE = 'wave'
    WIND = 'wind'
    TIDES = 'tides'
    WEATHER = 'weather'

In [28]:
@dataclass
class SlApiParams(Enum):
    SPOT_ID = 'spotId'
    DAYS = 'days'
    INTERVAL_HOURS = 'intervalHours'
    MAX_HEIGHTS = 'maxHeights'
    SDS = 'sds'
    ACCESSTOKEN = 'accesstoken'

In [29]:
def fetch_from_sl_api(endpoint: SlApiEndpoints, param_type: SlApiParams, param: str):
    base_url = "https://services.surfline.com/kbyg/spots/forecasts"
    res = requests.get(f"{base_url}/{endpoint}", params={param_type: param})
    data = res.json()
    return data

In [30]:
test_res = fetch_from_sl_api(SlApiEndpoints.RATING.value, SlApiParams.SPOT_ID.value, param=jetty_id)

In [31]:
spot_ratings = []
for spot_id, spot_name in df[['ids', 'names']][:3].values:
    res = requests.get(f"{base}/rating", params={'spotId': spot_id})
    data = res.json()
    cull_extra_days(data)
    current_date = pendulum.now("utc")
    utc_date = current_date.strftime("%Y-%m-%d")
    data['spot_id'] = spot_id
    data['spot_name'] = spot_name
    data['utc_fetch_date'] = utc_date
    spot_ratings.append(data)
    # time.sleep()

In [32]:
pendulum.from_timestamp(rating_json['data']['rating'][0]['timestamp'], 'UTC')

DateTime(2024, 6, 10, 4, 0, 0, tzinfo=Timezone('UTC'))

In [33]:
ratings_df = pd.json_normalize(spot_ratings, record_path=['data', 'rating'], meta=['spot_id', 'spot_name', 'utc_fetch_date'] )

In [34]:
ratings_df

,timestamp,utcOffset,rating.key,rating.value,spot_id,spot_name,utc_fetch_date
0,1717995600,-5,VERY_POOR,0,584204204e65fad6a7709b5d,Dauphin Island,2024-06-10
1,1717999200,-5,VERY_POOR,0,584204204e65fad6a7709b5d,Dauphin Island,2024-06-10
2,1718002800,-5,VERY_POOR,0,584204204e65fad6a7709b5d,Dauphin Island,2024-06-10
3,1718006400,-5,VERY_POOR,0,584204204e65fad6a7709b5d,Dauphin Island,2024-06-10
4,1718010000,-5,VERY_POOR,0,584204204e65fad6a7709b5d,Dauphin Island,2024-06-10
...,...,...,...,...,...,...,...
67,1718064000,-5,VERY_POOR,0,584204204e65fad6a7709b62,Alabama Point,2024-06-10
68,1718067600,-5,VERY_POOR,0,584204204e65fad6a7709b62,Alabama Point,2024-06-10
69,1718071200,-5,VERY_POOR,0,584204204e65fad6a7709b62,Alabama Point,2024-06-10
70,1718074800,-5,POOR,1,584204204e65fad6a7709b62,Alabama Point,2024-06-10


In [35]:
ratings_df['timestamp'] = ratings_df['timestamp'].apply(lambda x: pendulum.from_timestamp(x).to_datetime_string())

Alright, so it looks like each spot's forecast starts at 12am *local time*, with the timestamp for that time in unix. To figure out the flat `UTC` time for each spot you can just apply the `utcOffset` that is included in response. 

In [36]:
ratings_df['timestamp'] = pd.to_datetime(ratings_df['timestamp'])

In [37]:
ratings_df

,timestamp,utcOffset,rating.key,rating.value,spot_id,spot_name,utc_fetch_date
0,2024-06-10 05:00:00,-5,VERY_POOR,0,584204204e65fad6a7709b5d,Dauphin Island,2024-06-10
1,2024-06-10 06:00:00,-5,VERY_POOR,0,584204204e65fad6a7709b5d,Dauphin Island,2024-06-10
2,2024-06-10 07:00:00,-5,VERY_POOR,0,584204204e65fad6a7709b5d,Dauphin Island,2024-06-10
3,2024-06-10 08:00:00,-5,VERY_POOR,0,584204204e65fad6a7709b5d,Dauphin Island,2024-06-10
4,2024-06-10 09:00:00,-5,VERY_POOR,0,584204204e65fad6a7709b5d,Dauphin Island,2024-06-10
...,...,...,...,...,...,...,...
67,2024-06-11 00:00:00,-5,VERY_POOR,0,584204204e65fad6a7709b62,Alabama Point,2024-06-10
68,2024-06-11 01:00:00,-5,VERY_POOR,0,584204204e65fad6a7709b62,Alabama Point,2024-06-10
69,2024-06-11 02:00:00,-5,VERY_POOR,0,584204204e65fad6a7709b62,Alabama Point,2024-06-10
70,2024-06-11 03:00:00,-5,POOR,1,584204204e65fad6a7709b62,Alabama Point,2024-06-10


In [38]:
ratings_df['timestamp_utc'] = ratings_df.apply(lambda row: row['timestamp'] + pd.Timedelta(hours=row['utcOffset']), axis=1)

In [39]:
ratings_df

,timestamp,utcOffset,rating.key,rating.value,spot_id,spot_name,utc_fetch_date,timestamp_utc
0,2024-06-10 05:00:00,-5,VERY_POOR,0,584204204e65fad6a7709b5d,Dauphin Island,2024-06-10,2024-06-10 00:00:00
1,2024-06-10 06:00:00,-5,VERY_POOR,0,584204204e65fad6a7709b5d,Dauphin Island,2024-06-10,2024-06-10 01:00:00
2,2024-06-10 07:00:00,-5,VERY_POOR,0,584204204e65fad6a7709b5d,Dauphin Island,2024-06-10,2024-06-10 02:00:00
3,2024-06-10 08:00:00,-5,VERY_POOR,0,584204204e65fad6a7709b5d,Dauphin Island,2024-06-10,2024-06-10 03:00:00
4,2024-06-10 09:00:00,-5,VERY_POOR,0,584204204e65fad6a7709b5d,Dauphin Island,2024-06-10,2024-06-10 04:00:00
...,...,...,...,...,...,...,...,...
67,2024-06-11 00:00:00,-5,VERY_POOR,0,584204204e65fad6a7709b62,Alabama Point,2024-06-10,2024-06-10 19:00:00
68,2024-06-11 01:00:00,-5,VERY_POOR,0,584204204e65fad6a7709b62,Alabama Point,2024-06-10,2024-06-10 20:00:00
69,2024-06-11 02:00:00,-5,VERY_POOR,0,584204204e65fad6a7709b62,Alabama Point,2024-06-10,2024-06-10 21:00:00
70,2024-06-11 03:00:00,-5,POOR,1,584204204e65fad6a7709b62,Alabama Point,2024-06-10,2024-06-10 22:00:00


In [40]:
ratings_df.dtypes

timestamp         datetime64[ns]
utcOffset                  int64
rating.key                object
rating.value               int64
spot_id                   object
spot_name                 object
utc_fetch_date            object
timestamp_utc     datetime64[ns]
dtype: object

In [41]:
with SessionLocal() as db:
    matching_spots = []
    for spot in df['names']:
        stmt = text("""select * from spots where spot_name like :spot""")
        result = db.execute(stmt, {"spot": spot}).fetchall()
        if len(result) > 0:
            matching_spots.append(result)
    

In [42]:
len(matching_spots)

1302

In [43]:
jetty_waves = fetch_from_sl_api(SlApiEndpoints.WAVE.value, SlApiParams.SPOT_ID.value, jetty_id)

In [44]:
jetty_waves['associated']

{'units': {'temperature': 'F',
  'tideHeight': 'FT',
  'swellHeight': 'FT',
  'waveHeight': 'FT',
  'windSpeed': 'KTS',
  'pressure': 'MB'},
 'utcOffset': -4,
 'location': {'lon': -75.96648, 'lat': 36.83036135089083},
 'forecastLocation': {'lon': -75.947, 'lat': 36.847},
 'offshoreLocation': {'lon': -75.75, 'lat': 37},
 'runInitializationTimestamp': 1717956000}

In [45]:
jetty_waves['associated']['spotId'] = jetty_id

In [46]:
jetty_meta_df = pd.json_normalize(jetty_waves)
jetty_meta_df.drop(['permissions.violations', 'permissions.data', 'data.wave'], inplace=True, axis=1)

In [47]:
jetty_waves['data']['spotId'] = jetty_id

In [48]:
jetty_wave_df = pd.json_normalize(
    jetty_waves, record_path=["data", "wave"], meta=[["data", "spotId"]]
)
jetty_wave_df.drop("swells", inplace=True, axis=1)

In [49]:
jetty_wave_df.rename(columns={"power": "wave_power"}, inplace=True)

In [50]:
jetty_swell_df = pd.json_normalize(
    jetty_waves,
    record_path=['data', 'wave', 'swells'],
    meta=[['data', 'wave', 'timestamp'], ['data', 'spotId']]
)

In [51]:
jetty_swell_df['swells_idx'] = jetty_swell_df.groupby('data.wave.timestamp').cumcount()

In [52]:
jetty_swell_df.head()

,height,period,impact,power,direction,directionMin,optimalScore,data.wave.timestamp,data.spotId,swells_idx
0,0.00000,0,0.0000,0.00000,0.00,0.000,0,1717992000,584204214e65fad6a7709ce7,0
1,1.37795,6,0.6371,7.59931,138.16,128.505,0,1717992000,584204214e65fad6a7709ce7,1
2,0.26247,3,0.0000,0.00000,282.20,274.455,0,1717992000,584204214e65fad6a7709ce7,2
3,0.45932,9,0.3025,0.93201,74.21,69.015,0,1717992000,584204214e65fad6a7709ce7,3
4,0.39370,3,0.0333,0.00732,19.05,12.530,0,1717992000,584204214e65fad6a7709ce7,4


In [53]:
jetty_swell_df.rename({"power": 'swell_power'}, inplace=True)

In [54]:
jetty_swell_df['data.wave.timestamp'].value_counts()

1717992000    6
1717995600    6
1718308800    6
1718305200    6
1718301600    6
             ..
1718118000    6
1718114400    6
1718110800    6
1718107200    6
1718420400    6
Name: data.wave.timestamp, Length: 120, dtype: int64

In [55]:
jetty_meta_df.head()

,associated.units.temperature,associated.units.tideHeight,associated.units.swellHeight,associated.units.waveHeight,associated.units.windSpeed,associated.units.pressure,associated.utcOffset,associated.location.lon,associated.location.lat,associated.forecastLocation.lon,associated.forecastLocation.lat,associated.offshoreLocation.lon,associated.offshoreLocation.lat,associated.runInitializationTimestamp,associated.spotId
0,F,FT,FT,FT,KTS,MB,-4,-75.96648,36.830361,-75.947,36.847,-75.75,37,1717956000,584204214e65fad6a7709ce7


In [56]:
jetty_wave_df.head()

,timestamp,probability,utcOffset,wave_power,surf.min,surf.max,surf.plus,surf.humanRelation,surf.raw.min,surf.raw.max,surf.optimalScore,data.spotId
0,1717992000,100.000000,-4,8.53864,1,2,False,Knee to thigh,0.05070,1.03579,0,584204214e65fad6a7709ce7
1,1717995600,100.000000,-4,4.97555,1,2,False,Knee to thigh,0.05369,1.04176,0,584204214e65fad6a7709ce7
2,1717999200,100.000000,-4,9.54463,1,2,False,Knee to thigh,0.05667,1.04772,0,584204214e65fad6a7709ce7
3,1718002800,96.666667,-4,5.84679,1,2,False,Knee to thigh,0.05965,1.05369,0,584204214e65fad6a7709ce7
4,1718006400,96.666667,-4,5.72084,1,2,False,Knee to thigh,0.02983,1.02684,0,584204214e65fad6a7709ce7


In [57]:
jetty_swell_df.head(n=10)

,height,period,impact,power,direction,directionMin,optimalScore,data.wave.timestamp,data.spotId,swells_idx
0,0.00000,0,0.0000,0.00000,0.00,0.000,0,1717992000,584204214e65fad6a7709ce7,0
1,1.37795,6,0.6371,7.59931,138.16,128.505,0,1717992000,584204214e65fad6a7709ce7,1
2,0.26247,3,0.0000,0.00000,282.20,274.455,0,1717992000,584204214e65fad6a7709ce7,2
3,0.45932,9,0.3025,0.93201,74.21,69.015,0,1717992000,584204214e65fad6a7709ce7,3
4,0.39370,3,0.0333,0.00732,19.05,12.530,0,1717992000,584204214e65fad6a7709ce7,4
5,0.00000,0,0.0000,0.00000,0.00,0.000,0,1717992000,584204214e65fad6a7709ce7,5
6,0.95144,6,0.3972,2.56889,124.09,118.470,0,1717995600,584204214e65fad6a7709ce7,0
7,0.00000,0,0.0000,0.00000,0.00,0.000,0,1717995600,584204214e65fad6a7709ce7,1
8,0.19685,3,0.0000,0.00000,281.54,274.100,0,1717995600,584204214e65fad6a7709ce7,2
9,0.45932,9,0.2371,0.73051,75.04,69.505,0,1717995600,584204214e65fad6a7709ce7,3


In [58]:
combined_waves_df = pd.merge(
    jetty_wave_df,
    jetty_swell_df,
    how="inner",
    left_on=["timestamp", "data.spotId"],
    right_on=["data.wave.timestamp", 'data.spotId'],
)

In [59]:
len(combined_waves_df)

720

In [60]:
combined_waves_df

,timestamp,probability,utcOffset,wave_power,surf.min,surf.max,surf.plus,surf.humanRelation,surf.raw.min,surf.raw.max,...,data.spotId,height,period,impact,power,direction,directionMin,optimalScore,data.wave.timestamp,swells_idx
0,1717992000,100.0,-4,8.53864,1,2,False,Knee to thigh,0.05070,1.03579,...,584204214e65fad6a7709ce7,0.00000,0,0.0000,0.00000,0.00,0.000,0,1717992000,0
1,1717992000,100.0,-4,8.53864,1,2,False,Knee to thigh,0.05070,1.03579,...,584204214e65fad6a7709ce7,1.37795,6,0.6371,7.59931,138.16,128.505,0,1717992000,1
2,1717992000,100.0,-4,8.53864,1,2,False,Knee to thigh,0.05070,1.03579,...,584204214e65fad6a7709ce7,0.26247,3,0.0000,0.00000,282.20,274.455,0,1717992000,2
3,1717992000,100.0,-4,8.53864,1,2,False,Knee to thigh,0.05070,1.03579,...,584204214e65fad6a7709ce7,0.45932,9,0.3025,0.93201,74.21,69.015,0,1717992000,3
4,1717992000,100.0,-4,8.53864,1,2,False,Knee to thigh,0.05070,1.03579,...,584204214e65fad6a7709ce7,0.39370,3,0.0333,0.00732,19.05,12.530,0,1717992000,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,1718420400,NaN,-4,30.99348,2,3,False,Thigh to waist,1.08268,2.06693,...,584204214e65fad6a7709ce7,0.00000,0,0.0000,0.00000,0.00,0.000,0,1718420400,1
716,1718420400,NaN,-4,30.99348,2,3,False,Thigh to waist,1.08268,2.06693,...,584204214e65fad6a7709ce7,0.00000,0,0.0000,0.00000,0.00,0.000,0,1718420400,2
717,1718420400,NaN,-4,30.99348,2,3,False,Thigh to waist,1.08268,2.06693,...,584204214e65fad6a7709ce7,0.82021,8,0.2523,1.82996,107.02,100.915,0,1718420400,3
718,1718420400,NaN,-4,30.99348,2,3,False,Thigh to waist,1.08268,2.06693,...,584204214e65fad6a7709ce7,0.39370,16,0.2101,1.45973,102.97,99.190,0,1718420400,4


In [61]:
combined_df = pd.merge(jetty_meta_df, combined_waves_df, how='cross')

In [62]:
combined_df

,associated.units.temperature,associated.units.tideHeight,associated.units.swellHeight,associated.units.waveHeight,associated.units.windSpeed,associated.units.pressure,associated.utcOffset,associated.location.lon,associated.location.lat,associated.forecastLocation.lon,...,data.spotId,height,period,impact,power,direction,directionMin,optimalScore,data.wave.timestamp,swells_idx
0,F,FT,FT,FT,KTS,MB,-4,-75.96648,36.830361,-75.947,...,584204214e65fad6a7709ce7,0.00000,0,0.0000,0.00000,0.00,0.000,0,1717992000,0
1,F,FT,FT,FT,KTS,MB,-4,-75.96648,36.830361,-75.947,...,584204214e65fad6a7709ce7,1.37795,6,0.6371,7.59931,138.16,128.505,0,1717992000,1
2,F,FT,FT,FT,KTS,MB,-4,-75.96648,36.830361,-75.947,...,584204214e65fad6a7709ce7,0.26247,3,0.0000,0.00000,282.20,274.455,0,1717992000,2
3,F,FT,FT,FT,KTS,MB,-4,-75.96648,36.830361,-75.947,...,584204214e65fad6a7709ce7,0.45932,9,0.3025,0.93201,74.21,69.015,0,1717992000,3
4,F,FT,FT,FT,KTS,MB,-4,-75.96648,36.830361,-75.947,...,584204214e65fad6a7709ce7,0.39370,3,0.0333,0.00732,19.05,12.530,0,1717992000,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,F,FT,FT,FT,KTS,MB,-4,-75.96648,36.830361,-75.947,...,584204214e65fad6a7709ce7,0.00000,0,0.0000,0.00000,0.00,0.000,0,1718420400,1
716,F,FT,FT,FT,KTS,MB,-4,-75.96648,36.830361,-75.947,...,584204214e65fad6a7709ce7,0.00000,0,0.0000,0.00000,0.00,0.000,0,1718420400,2
717,F,FT,FT,FT,KTS,MB,-4,-75.96648,36.830361,-75.947,...,584204214e65fad6a7709ce7,0.82021,8,0.2523,1.82996,107.02,100.915,0,1718420400,3
718,F,FT,FT,FT,KTS,MB,-4,-75.96648,36.830361,-75.947,...,584204214e65fad6a7709ce7,0.39370,16,0.2101,1.45973,102.97,99.190,0,1718420400,4


In [75]:
class SurflineSpots:
    def __init__(self):
        self.states = []
        self.state_ids = []
        self.state_urls = []
        self.state_data = []
        self.county_data = []
        self.region_data = [] 
        self.spot_ids = []
        self.spot_names = []
        self.spot_address = []
        self.spot_lon = []
        self.spot_lat = []
        self.spot_urls = []

    def _update_states(self):
        response = requests.get(
            "https://services.surfline.com/taxonomy?type=taxonomy&id=58f7ed51dadb30820bb3879c&maxDepth=0"
        )
        json_data = response.json()
        json_contains = json_data["contains"]
        for x in json_contains:
            self.states.append(x["name"])
            self.state_ids.append(x["_id"])
        
        for state_id in self.state_ids:
            self.state_urls.append("https://services.surfline.com/taxonomy?type=taxonomy&id=" + state_id + "&maxDepth=0")
        logging.info(f"{len(self.state_urls)}")


    async def fetch_url(self, url, session):
        async with session.get(url) as response:
            return await response.json()


    async def fetch_all_urls(self, target):
        data = []
        async with aiohttp.ClientSession() as session:
            tasks = []
            for url in target:
                tasks.append(self.fetch_url(url, session))
            data = await asyncio.gather(*tasks)
        return data


    def update_data(self, data_target: List[str], attr_target):
        data = asyncio.run(self.fetch_all_urls(data_target))
        setattr(self, attr_target, data)
    

    def process_spots(self):
        if len(self.states) == 0:
            self._update_states() 
        
        self.update_data(self.state_urls, "state_data")
        logging.info("spots")



        county_ids = []    
        for state in self.state_data:
            state_contains = state['contains']
            for y in state_contains:
                county_ids.append(y['_id'])
        
        county_urls = []
        for county_id in county_ids:
            county_urls.append("https://services.surfline.com/taxonomy?type=taxonomy&id=" + county_id + "&maxDepth=0")

        self.update_data(county_urls, "county_data")

        region_ids = []
        region_names = []
        for county in self.county_data:
            county_contains = county['contains']
            for z in county_contains:
                region_ids.append(z['_id'])
                region_names.append(z['name'])

        region_urls = []
        for region_id in region_ids:
            region_urls.append("https://services.surfline.com/taxonomy?type=taxonomy&id=" + region_id + "&maxDepth=0")

        self.update_data(region_urls, "region_data")


        for region in self.region_data:
            region_contains = region['contains']
            if len(region_contains) == 0:
                self.spot_ids.append(region.get("spot", ""))
                self.spot_names.append(region.get("name", ""))
                self.spot_address.append("")
                region_associated = region['associated']
                region_links = region_associated['links']
                region_location = region['location']
                region_coordinates = region_location['coordinates']
                self.spot_lon.append(region_coordinates[0])
                self.spot_lat.append(region_coordinates[1])
                for i in region_links:
                    if i['key'] == "www":
                        self.spot_urls.append(i['href'])

        df = pd.DataFrame({"ids": self.spot_ids, "names": self.spot_names, "lon": self.spot_lon, "lat": self.spot_lat, "urls": self.spot_urls})
        return df 

In [76]:
spots = SurflineSpots()

In [77]:
df = spots.process_spots()

INFO:root:30
INFO:root:spots


In [78]:
df

,ids,names,lon,lat,urls
0,,Town of Seabrook,-70.860840,42.886900,https://www.surfline.com/surf-reports-forecast...
1,,Town of Rye,-70.721080,43.001790,https://www.surfline.com/surf-reports-forecast...
2,,Town of North Hampton,-70.828540,42.978520,https://www.surfline.com/surf-reports-forecast...
3,,North Hampton,-70.829780,42.972590,https://www.surfline.com/surf-reports-forecast...
4,,Town of Hampton,-70.834750,42.938360,https://www.surfline.com/surf-reports-forecast...
...,...,...,...,...,...
1301,584204204e65fad6a77094ee,Holland Light House,-86.217218,42.772947,https://www.surfline.com/surf-report/holland-l...
1302,584204204e65fad6a77094e7,Norton Shores,-86.298230,43.164465,https://www.surfline.com/surf-report/norton-sh...
1303,584204204e65fad6a77094eb,Saugatuck Dunes,-86.209260,42.701059,https://www.surfline.com/surf-report/saugatuck...
1304,584204204e65fad6a77094f6,Gordon Beach,-86.705452,41.829739,https://www.surfline.com/surf-report/gordon-be...


In [66]:
df

,ids,names,lon,lat,urls
0,,US Virgin Islands,-64.98348,18.34829,https://www.surfline.com/surf-reports-forecast...


In [ ]:
def fetch_sl_spots():
    response = requests.get(
        "https://services.surfline.com/taxonomy?type=taxonomy&id=58f7ed51dadb30820bb3879c&maxDepth=0"
    )
    json_data = response.json()
    json_contains = json_data["contains"]
    states = []
    state_ids = []
    state_urls = []
    for x in json_contains:
        states.append(x["name"])
        state_ids.append(x["_id"])
    

    for state_id in state_ids:
        state_urls.append("https://services.surfline.com/taxonomy?type=taxonomy&id=" + state_id + "&maxDepth=0")

In [ ]:
def transform_sl_wave_data(data: Dict) -> pd.DataFrame:
    if not data:
        raise ValueError("Data is empty")

    data["associated"]["spotId"] = jetty_id
    meta_df = pd.json_normalize(jetty_waves)
    jetty_meta_df.drop(['permissions.violations', 'permissions.data', 'data.wave'], inplace=True, axis=1)   